<font size=8 face="Courier"><b>Final Neuron Layout Source Code

This is the code use to create the plots in the notebook, `Final_Neuron_Layout.ipynb`. The purpose of the notebook is to help you understand how the location of a neuron relates to it's firing pattern. The most important part of the code are the functions `neuronLayout` and `spikeRaster`, located inside of the **Functions->Neuron Page** section. This is the code use to create the plots.

If you'd like to adapt this code for your own purposes, the key take home is that you can identify the x/y location of a neuron that you are looking at on the spikeraster by using the object, `sd.neuron_data[0]`. In this example. `sd` equals one of the datasets, `data_before, data_stim, data_after`. `sd.neuron_data[0][{neuron_number}]["position"]` provides the x/y location of a neuron, where the *"neuron number"* corresponds to the neuron's row number on the spike raster.

# <font color="gray">Import Stuff

In [ ]:
import numpy as np                                                    # Packages for data analysis
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
from scipy.ndimage import gaussian_filter1d
import braingeneers                                                   # Braingeneers code
from braingeneers.analysis.analysis import SpikeData, read_phy_files
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets 
import ipywidgets as ipw
from IPython.display import HTML, display, Javascript, clear_output

# <font color="green">Widgets


This section creates all the little interactive tools used to create the plots. It's not that necessary that you understand this code. The most important thing is that you understand how the values of the widgets (literally called `.values` in the later code) relate to variables that you can manipulate to create the plots.

## <font color="green">Start Page

simple start message

In [ ]:
start_message= ipw.HTML(value="<h4>Select the <i>group</i> and <i>file</i> that you would like to analyze.</h4>")
start_message

`start_group_name` is used to set the `group_num` to ensure you download the right dataset with the `loadData` function

In [ ]:
start_group_name= ipw.Dropdown( options=['2', '3', '4', '5', '6'], value='2', description='Group:')
start_group_name

`start_data_name` selects the dataset you wish to analyze, `data_before`, `data_stim`, or `data_after`. This dataset is saved as the variable `sd`

In [ ]:
start_data_name= ipw.Dropdown( options=['data_before', 'data_stim', 'data_after'], value='data_after', description='File:')
start_data_name

`start_button` simply brings you to the next window.

In [ ]:
start_button = ipw.Button(description="Submit")

`start` contains all the code to create the Start Page

In [ ]:
start = ipw.VBox([start_message, start_group_name, start_data_name, start_button])
start

## <font color="green">Neuron Page

Simple message to user for neuron page

In [ ]:
neuron_message= ipw.HTML(value="<h4>Select the <i>neuron</i> you are interested in.</h4>")
neuron_message

`neuron_num` selects the neuron you want to analyze. This corresponds to the row number of the spike raster. It also corresponds to the x/y position of the neuron, `sd.neuron_data[0][{neuron_number}]["position"]`

In [ ]:
neuron_num=ipw.IntSlider(description="Neuron:")
neuron_num

simple button to move you to the next page

In [ ]:
neuron_button = ipw.Button(description="Submit")

In [ ]:
neuron = ipw.VBox([neuron_message, neuron_num, neuron_button])
neuron

## <font color="green">Plots page

There is only one button on the page that presents the plot, this is `reset_button`. It brings you back to the start so that you can analyze other datasets and neurons.

In [ ]:
reset_button = ipw.Button(description="Reset")

# <font color="blue">Functions

These are the functions used to move between pages and create the plots. Notice that these functions only run when one of the buttons is clicked.  It's less important that you understand the code fog moving between pages, instead, I would focus on the helper functions used to retreive the dataset and create the plots.

## <font color="blue">Start Page

These are the functions that run when the `start_button` is clicked on the Start Page

### <font color="blue">Helper : Load Data <font color="gray"><small>Same Code as before

Note, this is the same code used in the `Final_Source_Code.ipynb`

<font color="blue">loadData</font> loads all the necesary data for analyzing one group. It's input is the number of the groups whose data you wish to analyze. It's "output" are the following global variables containing datasets used by later functions.
* `group_num` - the number of the group whose data we are looking at
* `data_before` - data recorded before stimulation
* `data_stim` - uncurated stimulation pattern
* `data_stim_bad` - curated stimulation pattern, it's "bad" because you can't see the stim patterns
* `data_after` - data recorded right after stimulation

In [ ]:
def loadData(group_number):
    global group_num; global data_before; global data_stim; global data_stim_bad; global data_after 
    
    print(f"loading data for group {str(group_number)}")        # Set global variables used by later functions
    group_num = group_number
    data_before = read_phy_files(f"./data/curated/grp{str(group_num)}_before.zip")
    data_stim = read_phy_files(f"./data/uncurated/grp{str(group_num)}_stim.zip")
    data_stim_bad = read_phy_files(f"./data/curated/grp{str(group_num)}_stim.zip")
    data_after = read_phy_files(f"./data/curated/grp{str(group_num)}_after.zip")
    print("done")

### <font color="blue">main function

This is the function that runs when the `start_button` is clicked. It shows the next page, which is the Neuron Page

In [ ]:
def showNeuronPage(b):
    global sd
    loadData(int(start_group_name.value))
    sds = {"data_before":data_before, "data_stim":data_stim, "data_after":data_after}    # 3 datasets data_name is allowed to be
    sd = sds[start_data_name.value]                                                      # set dataset we are interested in to sd                                                
    
    neuron_num.max = len(sd.neuron_data[0]) - 1                                          # set number of neurons to that of the dataset
    clear_output()
    display(neuron)
start_button.on_click(showNeuronPage)

## <font color="blue">Neuron Page

This is the function that runs when the `neuron_button` is clicked. It shows the next page, which is the Plots Page

### <font color="blue">Helper: neuron layout

This is the code used to create the 2D layout of the organoids, electrodes, neurons, and stimulation locations.

In [ ]:
def neuronLayout():
    
    # Stim electrodes for each group, and electrodes x/y locations
    grp_stim_electrodes = {2:[15569], 3:[15569, 13810], 4:[15569, 13810], 5:[15569, 6705], 6:[15569, 13810, 19971]}
    stim_xy = { 19971:[2992.5,1575.0], 13810:[2975.0,1085.0], 15569:[2957.5,1225.0], 6705:[1837.5,525.0] }
                                                                 
    # Add background image
    plt.figure(figsize=(15,10))
    img = plt.imread("data/Organoids_Closeup.jpeg")
    plt.imshow(img,  extent=[0, 3850, 0, 2100] ) #, #interpolation='nearest', aspect='auto')
    
    # Plot electrodes
    electrode_mapping = pd.DataFrame.from_dict( sd.metadata[0], orient="index", columns=['x','y']  ) 
    x = electrode_mapping.x.values                                                       # create electrode mapping from saved metadata
    y = electrode_mapping.y.values
    plt.scatter(x,y,s=4, c='darkorange')                                                                 # plot all electrodes
    
    # Plot neurons
    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():                                            # Get X/Y locations of neurons
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    plt.scatter(neuron_x,neuron_y, c='magenta', s=100 )                                                # plot all neurons
    
    # plot stim electrodes
    stim_x = []
    stim_y = []
    for electrode in grp_stim_electrodes[ group_num ]:
        stim_x.append( stim_xy[electrode][0] )
        stim_y.append( stim_xy[electrode][1] ) 
    plt.scatter(stim_x,stim_y, c='turquoise', alpha=.8, s=450 ) 
    
    # Plot selected neuron
    plt.scatter( sd.neuron_data[0][neuron_num.value]["position"][0], sd.neuron_data[0][neuron_num.value]["position"][1], c='blue', s=150 )   
    
    #add legend, axises limits, labels,  and title
    legend_elements = [Patch("darkorange","darkorange"), Patch("magenta","magenta"), Patch("turquoise","turquoise"), Patch("blue","blue") ]   # Create colors in legend
    plt.legend(legend_elements, ["Electrode","Other Neuron", "Stimulation", "Selected Neuron"])       # Add legend
    plt.xlim(0, 3850)
    plt.ylim(0, 2100)
    plt.xlabel('um')                                                                     # add axises and title
    plt.ylabel('um')
    plt.title(f"Neuron & Electrode Layout")
    plt.show()    

### <font color="blue">Helper: Spike Raster

This is the code used to create the spike raster.

In [ ]:
def spikeRaster():
    # Basic Scatter Plot
    plt.figure(figsize=(15,10))
    idces, times = sd.idces_times()
    plt.scatter(times/1000,idces,marker='|',s=1, c="r")   # Creates spike raster

    # Plot Neuron of interest
    neuron_vals = np.where(idces == neuron_num.value )[0]
    plt.scatter( times[neuron_vals]/1000, idces[neuron_vals], marker='|',s=2, c="b")  
    
    # Add labels to plot
    plt.title( "Spike Raster" )
    plt.xlabel("Time(s)")
    plt.ylabel('Unit #')
    plt.show

### <font color="blue">main function

This is the function that runs when the `neuron_button` is clicked. It shows the next page, which is the Plots of the data

In [ ]:
def showPlots(b):
    clear_output()
    display(ipw.HTML(f"<h4>Displaying Neuron {str(neuron_num.value)} for Group {start_group_name.value}- {start_data_name.value}</h4>"))
    display(reset_button)
    neuronLayout()
    spikeRaster()
neuron_button.on_click(showPlots)

## <font color="blue">Plots Page

There is a `reset_button` on the Plots Page. When it is clicked, we bring the user back to the beginning so that they can analyze other datasets and neurons.

In [ ]:
def reset(b):
    clear_output()
    display(start)
reset_button.on_click(reset)